# Compute DWPCs for all metapaths

In [1]:
import lzma
import numpy
import pandas
import pathlib
import random
import scipy.sparse
import tqdm

import hetmech.degree_weight
from hetmech.hetmat import HetMat
import hetmech.matrix
import hetmech.degree_group

In [2]:
hetmat = HetMat('../../data/hetionet-v1.0.hetmat/')
hetmech.degree_weight.default_dwwc_method = hetmech.degree_weight.dwwc_recursive

All metapaths up to length 3 saving PC and DWPC matrices on disk -- actual hetnet.

In [3]:
metapaths = hetmat.metagraph.extract_metapaths('Disease', max_length=2)
# random.seed(1)
# metapaths = random.sample(metapaths, k=2)
len(metapaths)

88

In [4]:
def compute_save_dwpc(graph, metapath, damping=0.5, dense_threshold=1, dtype='float64', approx_ok=True):
    path = hetmat.get_path_counts_path(metapath, 'dwpc', damping, None)
    for inverse in (True, False):
        mp = metapath
        if inverse:
            mp = metapath.inverse
        for ext in ('.sparse.npz', '.npy'):
            path = pathlib.Path(str(path) + ext)
            if path.exists():
                try:
                    return graph.read_path_counts(mp, 'dwpc', damping)
                except BadZipfile:
                    continue
    row, col, dwpc_matrix = hetmech.degree_weight.dwpc(graph, metapath, damping=damping, 
                                                       dense_threshold=dense_threshold, dtype=dtype)
    path = hetmat.get_path_counts_path(metapath, 'dwpc', damping, None)
    hetmech.hetmat.save_matrix(dwpc_matrix, path)
    return row, col, dwpc_matrix

### Compute path counts

In [5]:
%%time
hetmat.path_counts_cache = hetmech.hetmat.PathCountPriorityCache(hetmat, allocate_GB=16)
for metapath in tqdm.tqdm(metapaths):
    row, col, dwpc_matrix = compute_save_dwpc(hetmat, metapath, damping=0, dtype='uint64')
    del dwpc_matrix

100%|██████████| 88/88 [00:00<00:00, 111.03it/s]

CPU times: user 752 ms, sys: 35.3 ms, total: 787 ms
Wall time: 796 ms


### Compute DWPCs

In [6]:
%%time
hetmat.path_counts_cache = hetmech.hetmat.PathCountPriorityCache(hetmat, allocate_GB=16)
mean_dwpcs = dict()
for metapath in tqdm.tqdm(metapaths):
    # Compute degree-weighted path counts
    row, col, dwpc_matrix = compute_save_dwpc(hetmat, metapath, damping=0.5)
    mean_dwpcs[(metapath, 'dwpc', 0.5)] = dwpc_matrix.mean()
    del dwpc_matrix

100%|██████████| 88/88 [00:01<00:00, 71.40it/s]

CPU times: user 1.17 s, sys: 48 ms, total: 1.22 s
Wall time: 1.24 s


In [7]:
hetmat.path_counts_cache = None

Read in matrices to get scaling factor (mean). Then compute all DGP for all permutations.

In [8]:
for name, permat in tqdm.tqdm(hetmat.permutations.items()):
    permat.path_counts_cache = hetmech.hetmat.PathCountPriorityCache(permat, allocate_GB=16)
    for metapath in metapaths:
        _, _, matrix = hetmech.degree_weight.dwpc(permat, metapath, damping=0.5, dense_threshold=0.7)
        source_deg_to_ind, target_deg_to_ind = hetmech.degree_group.metapath_to_degree_dicts(permat, metapath)
        scaler = mean_dwpcs[(metapath, 'dwpc', 0.5)]
        row_generator = hetmech.degree_group.generate_degree_group_stats(
            source_deg_to_ind, target_deg_to_ind, matrix, scale=True, scaler=scaler)
        degree_stats_df = (
            pandas.DataFrame(row_generator)
            .set_index(['source_degree', 'target_degree'])
        )
        path = permat.directory.joinpath('degree-grouped-path-counts', 'dwpc-0.5', f'{metapath}.tsv')
        path.parent.mkdir(parents=True, exist_ok=True)
        degree_stats_df.to_csv(path, sep='\t')
    permat.path_counts_cache = None

100%|██████████| 25/25 [09:59<00:00, 23.98s/it]


Create multi-permutation DGP summary metrics.

In [9]:
for metapath in metapaths:
    degree_stats_df = None
    for permat in hetmat.permutations.values():
        path = permat.directory.joinpath('degree-grouped-path-counts', 'dwpc-0.5', f'{metapath}.tsv')
        df = (
            pandas.read_table(path)
            .set_index(['source_degree', 'target_degree'])
            .assign(n_perms=1)
        )
        if degree_stats_df is None:
            degree_stats_df = df
        else:
            degree_stats_df += df
    degree_stats_df = hetmech.degree_group.compute_summary_metrics(degree_stats_df)
    degree_stats_df.drop(columns=['sum', 'sum_of_squares'], inplace=True)
    path = hetmat.directory.joinpath('adjusted-path-counts', 'dwpc-0.5', 'degree-grouped-permutations', f'{metapath}.tsv')
    path.parent.mkdir(parents=True, exist_ok=True)
    degree_stats_df.to_csv(path, sep='\t')

In [8]:
%%time

full_dwpc_df = None
for metapath in tqdm.tqdm(metapaths):
    stats_path = hetmat.directory.joinpath('adjusted-path-counts', 'dwpc-0.5', 'degree-grouped-permutations', 
                                           f'{metapath}.tsv')
    degree_stats_df = pandas.read_table(stats_path)

    dwpc_row_generator = hetmech.degree_group.dwpc_to_degrees(hetmat, metapath)
    dwpc_df = pandas.DataFrame(dwpc_row_generator)
    df = (dwpc_df
     .merge(degree_stats_df, on=['source_degree', 'target_degree'])
     .drop(columns=['source_degree', 'target_degree'])
     .rename(columns={'mean': 'p-dwpc', 'sd': 'sd-dwpc'}))
    if full_dwpc_df is None:
        full_dwpc_df = df
    else:
        full_dwpc_df = full_dwpc_df.append(df, ignore_index=True)
full_dwpc_df['r-dwpc'] = full_dwpc_df['dwpc'] - full_dwpc_df['p-dwpc']
full_dwpc_df['z-dwpc'] = full_dwpc_df['r-dwpc'] / full_dwpc_df['sd-dwpc']

100%|██████████| 88/88 [03:41<00:00,  2.51s/it]

CPU times: user 3min 26s, sys: 14.2 s, total: 3min 40s
Wall time: 3min 41s


In [9]:
full_dwpc_df.shape

(5806757, 16)

In [11]:
full_dwpc_df.head(2)

,dwpc,metapath,path-count,source_id,source_metanode,source_name,target_id,target_metanode,target_name,n,nnz,n_perms,p-dwpc,sd-dwpc,r-dwpc,z-dwpc
0,4.691174,DlA,1,DOID:0050156,Disease,idiopathic pulmonary fibrosis,UBERON:0001004,Anatomy,respiratory system,2325,16,25,0.032283,0.387904,4.658891,12.010430
1,4.304678,DlA,1,DOID:0050156,Disease,idiopathic pulmonary fibrosis,UBERON:0002012,Anatomy,pulmonary artery,1575,23,25,0.062862,0.516545,4.241816,8.211907


In [10]:
%%time
full_dwpc_df.to_csv("full_dwpc.tsv.xz", sep='\t', index=False, compression='xz')

CPU times: user 8min 6s, sys: 449 ms, total: 8min 7s
Wall time: 8min 7s
